In [1]:
# %%
import pandas as pd
from docxtpl import DocxTemplate
import os
from pathlib import Path


class SOPGenerator:
    """
    用于生成标准操作程序(SOP)文档的类
    
    该类封装了从Excel数据生成Word文档并转换为PDF的功能，
    遵循良好的编程实践原则。
    """
    
    def __init__(self, 
                 research_data_path='research_areas_journals_skills.xlsx',
                 universities_data_path='selected_30_universities.xlsx',
                 template_path='word_template.docx'):
        """
        初始化SOP生成器
        
        Parameters
        ----------
        research_data_path : str
            研究领域数据文件路径
        universities_data_path : str
            大学数据文件路径
        template_path : str
            Word模板文件路径
        """
        self.research_data_path = research_data_path
        self.universities_data_path = universities_data_path
        self.template_path = template_path
        
        # 创建输出目录
        self.word_output_dir = Path('word_output')
        self.pdf_output_dir = Path('pdf_output')
        self._create_output_directories()
        
        # 加载数据和模板
        self.research_df = self._load_research_data()
        self.universities_df = self._load_universities_data()
        self.template = self._load_template()

    def _create_output_directories(self):
        """创建输出目录"""
        self.word_output_dir.mkdir(exist_ok=True)
        self.pdf_output_dir.mkdir(exist_ok=True)

    def _load_research_data(self):
        """加载研究领域数据"""
        if not Path(self.research_data_path).exists():
            raise FileNotFoundError(f"研究数据文件不存在: {self.research_data_path}")
        return pd.read_excel(self.research_data_path)

    def _load_universities_data(self):
        """加载大学数据"""
        if not Path(self.universities_data_path).exists():
            raise FileNotFoundError(f"大学数据文件不存在: {self.universities_data_path}")
        return pd.read_excel(self.universities_data_path)

    def _load_template(self):
        """加载Word模板"""
        if not Path(self.template_path).exists():
            raise FileNotFoundError(f"模板文件不存在: {self.template_path}")
        return DocxTemplate(self.template_path)

    def _generate_safe_filename(self, research_area, university):
        """
        生成安全的文件名
        
        Parameters
        ----------
        research_area : str
            研究领域
        university : str
            大学名称
            
        Returns
        -------
        str
            安全的文件名
        """
        safe_university = ''.join(c if c.isalnum() else '_' for c in university)
        safe_research = ''.join(c if c.isalnum() else '_' for c in research_area)
        return f"{safe_research}_{safe_university}"

    def _render_and_save_document(self, research_area, university, journals, skills):
        """
        渲染并保存单个文档
        
        Parameters
        ----------
        research_area : str
            研究领域
        university : str
            大学名称
        journals : str
            期刊信息
        skills : str
            技能信息
            
        Returns
        -------
        str
            保存的文件路径
        """
        context = {
            'research_area': research_area,
            'university': university,
            'journals': journals,
            'skills': skills
        }
        
        # 渲染模板
        self.template.render(context)
        
        # 生成文件名并保存
        filename = self._generate_safe_filename(research_area, university)
        output_path = self.word_output_dir / f"{filename}.docx"
        self.template.save(output_path)
        
        return str(output_path)

    def generate_application_letters(self):
        """
        生成所有申请信
        
        Returns
        -------
        int
            生成的文件数量
        """
        file_count = 0
        generated_files = []
        
        # 遍历每个研究领域和大学的组合
        for _, research_row in self.research_df.iterrows():
            research_area = research_row['research_area']
            journals = research_row['journal']
            skills = research_row['skill']
            
            for _, university_row in self.universities_df.iterrows():
                university = university_row['university']
                
                # 生成文档
                file_path = self._render_and_save_document(
                    research_area, university, journals, skills
                )
                generated_files.append(file_path)
                file_count += 1
        
        print(f"\n成功生成 {file_count} 封申请信！")
        print(f"Word文档保存在 '{self.word_output_dir}' 文件夹中")
        
        return file_count, generated_files


def convert_docx_to_pdf(word_dir='word_output', pdf_dir='pdf_output'):
    """
    将Word文档转换为PDF格式
    
    Parameters
    ----------
    word_dir : str
        Word文档目录
    pdf_dir : str
        PDF输出目录
        
    Returns
    -------
    bool
        转换是否成功
    """
    try:
        from docx2pdf import convert
        
        # 确保输出目录存在
        Path(pdf_dir).mkdir(exist_ok=True)
        
        # 转换文档
        convert(word_dir, pdf_dir)
        print("所有文档已成功转换为PDF！")
        print(f"PDF文件保存在 '{pdf_dir}' 文件夹中")
        return True
        
    except ImportError:
        print("错误：未找到docx2pdf库，请安装: pip install docx2pdf")
        return False
    except Exception as e:
        print(f"PDF转换过程中出错：{e}")
        print("请确保已安装Microsoft Word，或尝试其他转换方法")
        return False


def main():
    """主函数"""
    try:
        # 初始化生成器
        generator = SOPGenerator()
        
        # 生成申请信
        file_count, _ = generator.generate_application_letters()
        
        # 转换为PDF
        if file_count > 0:
            convert_docx_to_pdf()
        else:
            print("没有生成任何文档，跳过PDF转换")
            
    except FileNotFoundError as e:
        print(f"文件错误：{e}")
    except Exception as e:
        print(f"程序执行过程中出现错误：{e}")


if __name__ == "__main__":
    main()


成功生成 90 封申请信！
Word文档保存在 'word_output' 文件夹中


  0%|          | 0/90 [00:00<?, ?it/s]

所有文档已成功转换为PDF！
PDF文件保存在 'pdf_output' 文件夹中
